# Generate Skill Cache

Helper tool for creating the implementation of `initializeSkillCache()` in `BB2020SkillSettings`.

In [ ]:
// Copy of the SkillType enum with some extra information
// 1. All `,` are remove
// 2. Skills not supported are marked with a * in front
// 3. Skill category is defined by // <Name>
// 4. :<value> is defining the default value, `null` is used if not set
// 5. The name inside "..." is used to generate the Class, some mistakes will be made here, but should be easy to fix by hand
public val skillDefinition = """
    // Agility
    CATCH("Catch")
    *DIVING_CATCH("Diving Catch")
    DIVING_TACKLE("Diving Tackle")
    DODGE("Dodge")
    *DEFENSIVE("Defensive")
    *JUMP_UP("Jump Up")
    LEAP("Leap")
    *SAFE_PAIR_OF_HANDS("Safe Pair of Hands")
    SIDESTEP("Sidestep")
    *SNEAKY_GIT("Sneaky Git")
    SPRINT("Sprint")
    SURE_FEET("Sure Feet")

    // General
    BLOCK("Block")
    *DAUNTLESS("Dauntless")
    *DIRTY_PLAYER("Dirty Player"):1
    *FEND("Fend")
    FRENZY("Frenzy")
    *KICK("Kick")
    PRO("Pro")
    *SHADOWING("Shadowing")
    *STRIP_BALL("Strip ball")
    SURE_HANDS("Sure Hands")
    TACKLE("Tackle")
    WRESTLE("Wrestle")

    // Mutations
    *BIG_HAND("Big Hand")
    *CLAWS("Claws")
    *DISTURBING_PRECENSE("Disturbing Precence")
    *EXTRA_ARMS("Extra Arms")
    *FOUL_APPEREANCE("Full Appereance")
    HORNS("Horns")
    *IRON_HARD_SKIN("Iron Hard Skin")
    *MONSTROUS_MOUTH("Monstrous Mouth")
    PREHENSILE_TAIL("Prehensile Tail")
    *TENTACLE("Tentacle")
    *TWO_HEADS("Two Heads")
    *VERY_LONG_LEGS("Very Long Legs")

    // Passing
    *ACCURATE("Accurate")
    *CANNONEER("Cannoneer")
    *CLOUD_BURSTER("Cloud Burster")
    *DUMP_OFF("Dump-off")
    *FUMBLEROOSKIE("Fumblerooskie")
    *HAIL_MARY_PASS("Hail Mary Pass")
    *LEADER("Leader")
    *NERVES_OF_STEEL("Nerves of Steel")
    *ON_THE_BALL("On the Ball")
    PASS("Pass")
    *RUNNING_PASS("Running Pass")
    *SAFE_PASS("Safe Pass")

    // Strength
    *ARM_BAR("Arm Bar")
    *BRAWLER("Brawler")
    BREAK_TACKLE("Break Tackle")
    *GRAB("Grab")
    *GUARD("Guard")
    *JUGGERNAUT("Juggernaut")
    MIGHTY_BLOW("Mighty Blow"):1
    MULTIPLE_BLOCK("Multiple Block")
    *PILE_DRIVER("Pile Driver")
    *STAND_FIRM("Stand Firm")
    *STRONG_ARM("Strong Arm")
    THICK_SKULL("Thick Skull")

    // Traits
    ANIMAL_SAVAGERY("Animal Savagery")
    *ANIMOSITY("Animosity")
    *ALWAYS_HUNGY("Always Hungy")
    *BALL_AND_CHAIN("Ball & Chain")
    *BOMBARDIER("Bombardier")
    BONE_HEAD("Bone Head")
    BLOOD_LUST("Blood Lust"):4
    BREATHE_FIRE("Breathe Fire")
    *CHAINSAW("Chainsaw")
    *DECAY("Decay")
    *HYPNOTIC_GAZE("Hypnotic Gaze")
    *KICK_TEAMMATE("Kick Team-mate")
    LONER("Loner"):4
    *NO_HANDS("No Hands")
    *PLAGUE_RIDDEN("Plague Ridden")
    *POGO_STICK("Pojo Stick")
    PROJECTILE_VOMIT("Projectile Vomit")
    REALLY_STUPID("Really Stupid")
    REGENERATION("Regeneration")
    *RIGHT_STUFF("Right Stuff")
    *SECRET_WEAPON("Secret Wapon")
    STAB("Stab")
    STUNTY("Stunty")
    *SWARMING("Swarming")
    *SWOOP("Swoop")
    *TAKE_ROOT("Take Root")
    TITCHY("Titchy")
    TIMMMBER("Timmm-ber!")
    *THROW_TEAMMATE("Throw Team-mate")
    UNCHANNELLED_FURY("Unchannelled Fury")

    // Special Rules
    SNEAKIEST_OF_THE_LOT("Sneakiest of the Lot")
""".trimIndent()


In [ ]:
import com.jervisffb.engine.rules.bb2020.skills.SkillCategory
import com.jervisffb.engine.rules.bb2020.skills.SkillType

// Parse skill descriptions
class SkillData(val type: SkillType, val defaultValue: Int?, val className: String?)
class SkillGroup(val category: SkillCategory, val skills: MutableList<SkillData> = mutableListOf())

val regex = "^\\*?([A-Z_]+)\\(\"(.*)\"\\)(:\\d)?$".toRegex()
val skillGroups = mutableListOf<SkillGroup>()
var currentGroup: SkillGroup? = null
val groups = skillDefinition.lines().forEach { line ->
    if (line.isBlank()) return@forEach
    if (line.startsWith("//")) {
        val category = SkillCategory.entries.first { it.description == line.removePrefix("//").trim() }
        currentGroup = SkillGroup(category)
        skillGroups.add(currentGroup!!)
    } else {
        val match = regex.matchEntire(line) ?: error("Did not match: '$line'")
        val disabled = line.startsWith("*")
        val skill = SkillData(
            type = SkillType.valueOf(match.groupValues[1]),
            defaultValue = match.groupValues.getOrNull(3)?.let { if (it.isBlank()) null else it.removePrefix(":").toInt() } ,
            className = if (!disabled) match.groupValues[2].replace(" ", "") else null
        )
        currentGroup!!.skills.add(skill)
    }
}

fun indent(size: Int): String {
    return " ".repeat(size*4)
}

val code = buildString {
    appendLine("override fun initializeSkillCache() {")
    for (category in skillGroups) {
        appendLine("${indent(1)}addCategory(SkillCategory.${category.category.name})")
    }
    appendLine(indent(1) + "SkillType.entries.forEach { type ->")
    appendLine(indent(2) + "when (type) {")
    for (category in skillGroups) {
        appendLine("")
        appendLine(indent(3) + "//")
        appendLine(indent(3) + "// ${category.category.description} Category")
        appendLine(indent(3) + "//")
        for (skill: SkillData in category.skills) {
            val disabled = (skill.className == null)
            val commentIfDisabled = if (disabled) "// " else ""
            appendLine(indent(3) + "SkillType.${skill.type.name} -> {")
            val defaultValue = if (skill.defaultValue != null) ", ${skill.defaultValue}" else ""
            val defaultValueArg = if (defaultValue.isNotEmpty()) "value" else "_"
            appendLine(indent(4) + commentIfDisabled + "addEntry(type, SkillCategory.${category.category.name}$defaultValue) { player, category, $defaultValueArg , expiresAt ->")
            val skillConstructor = if (skill.className != null) {
                if (defaultValue.isNotEmpty()) {
                    "${skill.className}(player, category, value, expiresAt)"
                } else {
                    "${skill.className}(player, category, expiresAt)"
                }
            } else {
                "TODO()"
            }
            appendLine(indent(5) + commentIfDisabled + skillConstructor)
            appendLine(indent(4) + commentIfDisabled + "}")
            appendLine(indent(3) + "}")
        }

    }
    appendLine(indent(2) + "}")
    appendLine(indent(1) + "}")
    appendLine("}")
}

code